## Feature Exploration & Cleaning: US County Area

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sqlite3

In [2]:
conn=sqlite3.connect('COVID19_county_data.db')
cursor= conn.cursor()

In [3]:
df=pd.read_csv('Census_County_area_v2.csv')

In [4]:
df.head()

,Areaname,FIPS,Total_Area_sq_mi_LND010190D,LND010200D,LND110180D,LND110190D,Land_Area_sq_mi_LND110200D,LND110210D,Water_Area_sq_miLND210190D,LND210200D
0,UNITED STATES,0,3787425.08,3794083.06,3539289.16,3536341.73,3537438.44,3531905.43,251083.35,256644.62
1,ALABAMA,1000,52422.94,52419.02,50767.18,50750.23,50744.00,50645.33,1672.71,1675.01
2,"Autauga, AL",1001,604.49,604.45,597.04,596.01,595.97,594.44,8.48,8.48
3,"Baldwin, AL",1003,2027.08,2026.93,1589.42,1596.53,1596.35,1589.78,430.55,430.58
4,"Barbour, AL",1005,904.59,904.52,883.89,885.00,884.90,884.88,19.59,19.61


In [5]:
new_cols=[col for col in df.columns if "sq_mi" in col]+['Areaname', 'FIPS']

In [6]:
df=df[new_cols]

In [7]:
df.head()

,Total_Area_sq_mi_LND010190D,Land_Area_sq_mi_LND110200D,Water_Area_sq_miLND210190D,Areaname,FIPS
0,3787425.08,3537438.44,251083.35,UNITED STATES,0
1,52422.94,50744.00,1672.71,ALABAMA,1000
2,604.49,595.97,8.48,"Autauga, AL",1001
3,2027.08,1596.35,430.55,"Baldwin, AL",1003
4,904.59,884.90,19.59,"Barbour, AL",1005


In [8]:
state_names=[name for name in df['Areaname'] if ',' not in name]

In [9]:
state_df=df[(df['Areaname'].isin(state_names))]

In [10]:
state_df.head()

,Total_Area_sq_mi_LND010190D,Land_Area_sq_mi_LND110200D,Water_Area_sq_miLND210190D,Areaname,FIPS
0,3787425.08,3537438.44,251083.35,UNITED STATES,0
1,52422.94,50744.00,1672.71,ALABAMA,1000
69,656424.14,571951.26,86050.59,ALASKA,2000
99,114006.26,113634.57,364.00,ARIZONA,4000
115,53182.36,52068.17,1107.07,ARKANSAS,5000


In [11]:
county_df=df[~(df['Areaname'].isin(state_names))]

In [12]:
county_df.head()

,Total_Area_sq_mi_LND010190D,Land_Area_sq_mi_LND110200D,Water_Area_sq_miLND210190D,Areaname,FIPS
2,604.49,595.97,8.48,"Autauga, AL",1001
3,2027.08,1596.35,430.55,"Baldwin, AL",1003
4,904.59,884.90,19.59,"Barbour, AL",1005
5,625.50,623.03,3.14,"Bibb, AL",1007
6,650.65,645.59,4.97,"Blount, AL",1009


In [13]:
county_df.dtypes

Total_Area_sq_mi_LND010190D    float64
Land_Area_sq_mi_LND110200D     float64
Water_Area_sq_miLND210190D     float64
Areaname                        object
FIPS                             int64
dtype: object

In [14]:
county_df.isna().sum()

Total_Area_sq_mi_LND010190D    0
Land_Area_sq_mi_LND110200D     0
Water_Area_sq_miLND210190D     0
Areaname                       0
FIPS                           0
dtype: int64

In [15]:
county_df['FIPS'] = county_df['FIPS'].apply(lambda x: str(int(x)).zfill(5))

/Users/michelewaters/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
county_df.to_sql('county_area', conn, index_label='id', if_exists='replace')